In [2]:
from PIL import Image
import numpy as np
import math

In [3]:
class camera:
    """ An class that contains all the data for a camera """
    def __init__(self, horizontal_fov: int, vertical_fov: int, resolution: tuple, location: tuple, direction: tuple, view_distance: tuple):
        self.horizontal_fov = horizontal_fov
        self.vertical_fov = vertical_fov
        self.resolution = resolution
        self.location = location
        self.direction = direction
        self.view_distance = view_distance

In [4]:
class mesh3D:
    """ An class that contains all the data for a mesh """
    def __init__(self, verticies: list, faces: list, center: tuple):
        self.verticies = verticies
        self.faces = faces
        self.center = center

In [5]:
def save_image(pixels: list, image_name: str):
    """takes in a numpy array of tuples of rgb values and converts it into a image then it saves the image as a file and displays the image"""
    pixels = list(zip(*pixels))[::-1]
    array = np.array(pixels, dtype=np.uint8)
    new_image = Image.fromarray(array)
    display(new_image)
    new_image.save(image_name + '.png')

In [6]:
def initialize_2dimage(dimensions: tuple, background_color: tuple) -> list:
    ''' Creates a three dimensional numpy array of a certain size that dimension1 x dimension2 x 3 to represent the RGB value at each point'''
    image = np.zeros((1000, 1000, 3), dtype=np.uint8)
    image[:, :, 0] = background_color[0] 
    image[:, :, 1] = background_color[1] 
    image[:, :, 2] = background_color[2] 
    return image

In [7]:
def line_2d(point_1: np.ndarray, point_2: np.ndarray):
    """ 
    Acts as a generator which yields the the cordinates of subsequent points on a line formed between two inputed points as tuples

    SPECIAL INFORMATION
    - Lines include both end points
    -only one pixel per line in the direction which moves the most over time 
        - EX: (0,0) to (5,20) there would only every be one box per y value
    """

    x1, y1 = point_1
    x2, y2 = point_2

    #preparation calculations (differences and direction)
    difference_x = abs(x2 - x1)
    difference_y = abs(y2 - y1)
    if x2 > x1:
        x_sign = 1
    else:
        x_sign = -1
    if y2 > y1:
        y_sign = 1
    else:
        y_sign = -1

    if difference_x > difference_y: # senario when the x value changes more than y in the line
        y_error = 2*difference_y - difference_x
        yield (int(x1),int(y1))
        while x1 != x2:
            x1 += x_sign
            if y_error >= 0:
                y1 += y_sign
                y_error -= 2 * difference_x
            y_error += 2 * difference_y 
            yield (int(x1),int(y1))

    else: # senarion when y changes more than x or both change the exact same amount
        x_error = 2*difference_x - difference_y
        yield (int(x1),int(y1))
        while y1 != y2:
            y1 += y_sign
            if x_error >= 0:
                x1 += x_sign
                x_error -= 2 * difference_y
            x_error += 2 * difference_x 
            yield (int(x1),int(y1))

In [8]:
def global_cords(center: np.ndarray, model_verticies: np.ndarray):
    """Will return location of a meshes verticies in global space from inputtings its center location and mesh information"""
    transformation_matrix = np.array([[1,0,0,center[0]],
                                      [0,1,0,center[1]],
                                      [0,0,1,center[2]],
                                      [0,0,0,1]])
    return [np.dot(transformation_matrix,vertex) for vertex in model_verticies]

In [12]:
def reorient_to_camera(self, cam: camera, global_verticies: np.ndarray):
    """reorients and rotates verticies to be aligned with the camera at (0,0) with the positive direction where the camera is looking"""
    translation_matrix = np.array([[1,0,0,-cam.location[0]],
                                    [0,1,0,-cam.location[1]],
                                    [0,0,1,-cam.location[2]],
                                    [0,0,0,1]])
    x_rotation_matrix = np.array([[1,0,0,0],
                                  [0,math.cos(math.radians(-cam.direction[0])),-math.sin(math.radians(-cam.direction[0])),0],
                                  [0,math.sin(math.radians(-cam.direction[0])),math.cos(math.radians(-cam.direction[0])),0],
                                  [0,0,0,1]])
    y_rotation_matrix = np.array([[math.cos(math.radians(-cam.direction[1])),0,math.sin(math.radians(-cam.direction[1])),0],
                                  [0,1,0,0],
                                  [-math.sin(math.radians(-cam.direction[1])),0,math.cos(math.radians(-cam.direction[1])),0],
                                  [0,0,0,1]])
    z_rotation_matrix = np.array([[math.cos(math.radians(-cam.direction[2])),-math.sin(math.radians(-cam.direction[2])),0,0],
                                  [math.sin(math.radians(-cam.direction[2])),math.cos(math.radians(-cam.direction[2])),0,0],
                                  [0,0,1,0],
                                  [0,0,0,1]])
    return [np.dot(np.dot(np.dot(np.dot(point,translation_matrix),x_rotation_matrix),y_rotation_matrix),z_rotation_matrix) for point in global_verticies] 

In [ ]:
############ POTENTIALLY REMOVE THIS IF CLIPPING IN THE RASTERIZATION FUNCTION WORKS WELL ENOUGH############
def clipping(cam: camera, oriented_to_camera_verticies: np.ndarray, faces: np.ndarray):
    ''' takes in verticies location in respect to the camera and runs through each face to determine if it is in the fov of the camera. If none of the face is in the camera it will get deleted'''
    remaining_faces = []
    for face in faces:
        for point_idx in face:
            point = oriented_to_camera_verticies[point_idx]
            if cam.horizontal_fov >= abs(math.degrees(math.tan(point[0]/point[2]))) and cam.vertical_fov >= abs(math.degrees(math.tan(point[1]/point[2]))) and cam.view_distance[1] >= point[2] >= cam.view_distance[0]:
                remaining_faces.append(face)
                break
    return remaining_faces

In [ ]:
def perspective_projection(cam: camera, oriented_to_camera_vertices: np.ndarray):

    projection_matrix = np.array([
        [cam.view_distance[0] / math.tan(math.radians(cam.horizontal_fov / 2)), 0, 0, 0],
        [0, cam.view_distance[0] / math.tan(math.radians(cam.vertical_fov / 2)), 0, 0],
        [0, 0, cam.view_distance[1] - cam.view_distance[0], -cam.view_distance[0] * cam.view_distance[0]],
        [0, 0, 1, 0]])

    points_in_2d = []
    for vertex in oriented_to_camera_vertices:
        homogeneous_point = np.dot(vertex, projection_matrix)
        w_reciprocal = 1.0 / homogeneous_point[3]

        # Avoid creating unnecessary NumPy array, use a temporary variable instead
        projected_point = [homogeneous_point[0] * w_reciprocal, homogeneous_point[1] * w_reciprocal, homogeneous_point[3], 1]
        points_in_2d.append(projected_point)

    return points_in_2d


In [ ]:
def back_culling(oriented_to_camera_verticies: np.ndarray, faces: np.ndarray):
    ''' takes in faces and verticies and returns a list of faces excluding any faces that would be looking away from the camera and occluded by other faces in front of it'''
    culled_faces = []
    for face in faces:
        ax, ay, az, _ = oriented_to_camera_verticies[face[0]]
        bx, by, bz, _  = oriented_to_camera_verticies[face[1]]
        cx, cy, cz, _  = oriented_to_camera_verticies[face[2]]
        #relies on any mesh textextures to have their faces to have vectors cross product face out of the mesh
        vector1 = np.array([bx-ax, by-ay, bz-az]) 
        vector2 = np.array([bx-cx, by-cy, bz-cz])
        angle = np.cross(vector1, vector2)
        if np.dot(angle, oriented_to_camera_verticies[face[1]][:2]) < 0:
            culled_faces.append(face)
    return culled_faces

In [ ]:
def screen_space(cam: camera, verticies: np.ndarray):
    ''' Takes in verticies after the perspective projection and adjusts them to the resolution of the screen '''
    return np.array([[round((vertex[0]+1)*cam.resolution[0]/2),round((vertex[1]+1)*cam.resolution[1]/2),vertex[2],1] for vertex in verticies])

Note to self use Cython to make the program run faster when I am done